**Zaliczenie części Wprowadzenie do Sparka**

In [63]:
!pip install pyspark

Zbior danych to oceny filmow. Dane pochodza z projektu https://movielens.org/
Wiecej informacji o danych mozna znalezc pod adresem: http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html

In [64]:
import pandas as pd

In [65]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataFrame_add').master('local[*]').getOrCreate()

data_path = '/'

Do rozważenia: Zalecanym formatem przechowywania danych jest Apache Parquet.
Wiecej o tym formacie danych mozna sie dowiedziec np. tutaj: https://parquet.apache.org/.
W skrocie, tak strona projektu podsumowuje misje stojaca za tym formatem:
"We created Parquet to make the advantages of compressed, efficient columnar data representation available to any project in the Hadoop ecosystem".

Przyjrzyj sie danym i przeanalizuj.

Sugestie:
1. Z jakich kolumn skladaja sie wczytane powyzej zbiory?
2. Informacje o ilu filmach znajduja sie w dostarczonym zbiorze?
3. Oceny ilu uzytkownikow znajduja sie w zbiorze?
4. Czy w zbiorze znajduja sie braki danych?
5. Ile filmow nie ma ocen? Ktore filmy nie maja ocen?
6. Ktory film ma najlepsza srednia ocen? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.
7. Jaki procent filmow ma tylko maksymalne oceny?
8. Ktory film na najwyzsza minimalna ocene? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.
9. Jaki jest rozklad ocen?
10. Ile jest filmow zaklasyfikowanych jako dokumentalne 'documentary'?
11. Ktory z filmow dokumentalnych z conajmniej 10 glosami ma najwysza srednia ocene?
12. ~~Jakie sa roznice pomiedzy liczba filmow w zbiorze z roku na rok? Zaloz, ze timestamp reprezentuje liczbe sekund od roku 1960.~~
13. Ile srednio kategorii przypisanych jest do 1 filmu? Ktory film ma najwiecej kategorii (+co to za kategorie)?

In [66]:
#Wczytanie zbiorów

links_df = spark.read.csv('links.csv', header=True, inferSchema=True)
movies_df = spark.read.csv('movies.csv', header=True, inferSchema=True)
ratings_df = spark.read.csv('ratings.csv', header=True, inferSchema=True)
tags_df = spark.read.csv('tags.csv', header=True, inferSchema=True)

In [67]:
#1. Z jakich kolumn składają się zbiory?
print(f"Columns in links.csv: {pd.read_csv('links.csv').columns.to_list()}")
print(f"Columns in movies.csv: {pd.read_csv('movies.csv').columns.to_list()}")
print(f"Columns in ratings.csv: {pd.read_csv('ratings.csv').columns.to_list()}")
print(f"Columns in tags.csv: {pd.read_csv('tags.csv').columns.to_list()}")

Columns in links.csv: ['movieId', 'imdbId', 'tmdbId']
Columns in movies.csv: ['movieId', 'title', 'genres']
Columns in ratings.csv: ['userId', 'movieId', 'rating', 'timestamp']
Columns in tags.csv: ['userId', 'movieId', 'tag', 'timestamp']


In [68]:
#2. Informacje o ilu filmach znajduja sie w dostarczonym zbiorze?
print(f"links.csv: {pd.read_csv('links.csv').shape}")
print(f"movies.csv: {pd.read_csv('movies.csv').shape}")
print(f"ratings.csv: {pd.read_csv('ratings.csv').shape}")
print(f"tags.csv: {pd.read_csv('tags.csv').shape}")

links.csv: (9742, 3)
movies.csv: (9742, 3)
ratings.csv: (100836, 4)
tags.csv: (3683, 4)


In [69]:
#3. Oceny ilu uzytkownikow znajduja sie w zbiorze?
pd.read_csv('ratings.csv')['userId'].drop_duplicates().count()

610

In [70]:
#4. Czy w zbiorze znajduja sie braki danych?
import os

csvs = [file for file in os.listdir('/content') if os.path.splitext(file)[1] == '.csv']

missing_values = 0
for csv in csvs:
  na = pd.read_csv(csv).isnull().sum().sum()
  if na != 0:
    print(f'{na} braków danych w {csv}')
  missing_values += na

print(f'Łącznie {missing_values} braków danych w całym zbiorze')

8 braków danych w links.csv
Łącznie 8 braków danych w całym zbiorze


In [71]:
#5. Ile filmow nie ma ocen? Ktore filmy nie maja ocen?

movies_with_ratings = movies_df.join(ratings_df, on=['movieId'], how='left')
movies_with_no_ratings = movies_with_ratings.filter(movies_with_ratings.rating.isNull())

#filmy bez ocen
print("Ilość filmów bez ocen:", movies_with_no_ratings.count())
movies_with_no_ratings.select("movieId", "title").show()

Ilość filmów bez ocen: 18
+-------+--------------------+
|movieId|               title|
+-------+--------------------+
|   1076|Innocents, The (1...|
|   2939|      Niagara (1953)|
|   3338|For All Mankind (...|
|   3456|Color of Paradise...|
|   4194|I Know Where I'm ...|
|   5721|  Chosen, The (1981)|
|   6668|Road Home, The (W...|
|   6849|      Scrooge (1970)|
|   7020|        Proof (1991)|
|   7792|Parallax View, Th...|
|   8765|This Gun for Hire...|
|  25855|Roaring Twenties,...|
|  26085|Mutiny on the Bou...|
|  30892|In the Realms of ...|
|  32160|Twentieth Century...|
|  32371|Call Northside 77...|
|  34482|Browning Version,...|
|  85565|  Chalet Girl (2011)|
+-------+--------------------+



In [72]:
#6. Ktory film ma najlepsza srednia ocen? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.

from pyspark.sql.functions import col, avg, count

#średnie oceny dla filmów
movie_ratings = ratings_df.groupBy("movieId").agg(avg("rating").alias("average_rating"), count("rating").alias("num_ratings"))

#film z najlepszą średnią oceną
best_rated_movie = movie_ratings.orderBy(col("average_rating").desc(), col("num_ratings").desc()).limit(1)

#znalezienie tytułu najlepiej ocenionego filmu
best_rated_movie.show()
best_rated_movie_title = best_rated_movie.join(movies_df, on=["movieId"], how="left")
best_rated_movie_title.select("title", "average_rating").show()

+-------+--------------+-----------+
|movieId|average_rating|num_ratings|
+-------+--------------+-----------+
|     53|           5.0|          2|
+-------+--------------+-----------+

+---------------+--------------+
|          title|average_rating|
+---------------+--------------+
|Lamerica (1994)|           5.0|
+---------------+--------------+



In [73]:
#7. Jaki procent filmów ma tylko maksymalne oceny?

from pyspark.sql.functions import max

#wczytanie zbiorów
movies_df = spark.read.csv('movies.csv', header=True, inferSchema=True)
ratings_df = spark.read.csv('ratings.csv', header=True, inferSchema=True)

#wykonanie joina
joined_df = ratings_df.join(movies_df, on="movieId", how="inner")

#wyświetlenie wyników
joined_df.show()

+-------+------+------+---------+--------------------+--------------------+
|movieId|userId|rating|timestamp|               title|              genres|
+-------+------+------+---------+--------------------+--------------------+
|      1|     1|   4.0|964982703|    Toy Story (1995)|Adventure|Animati...|
|      3|     1|   4.0|964981247|Grumpier Old Men ...|      Comedy|Romance|
|      6|     1|   4.0|964982224|         Heat (1995)|Action|Crime|Thri...|
|     47|     1|   5.0|964983815|Seven (a.k.a. Se7...|    Mystery|Thriller|
|     50|     1|   5.0|964982931|Usual Suspects, T...|Crime|Mystery|Thr...|
|     70|     1|   3.0|964982400|From Dusk Till Da...|Action|Comedy|Hor...|
|    101|     1|   5.0|964980868|Bottle Rocket (1996)|Adventure|Comedy|...|
|    110|     1|   4.0|964982176|   Braveheart (1995)|    Action|Drama|War|
|    151|     1|   5.0|964984041|      Rob Roy (1995)|Action|Drama|Roma...|
|    157|     1|   5.0|964984100|Canadian Bacon (1...|          Comedy|War|
|    163|   

In [74]:
#7. Jaki procent filmow ma tylko maksymalne oceny?

from pyspark.sql.functions import col, avg

#wczytanie potrzebnych zbiorów
movies_df = spark.read.csv('movies.csv', header=True, inferSchema=True)
ratings_df = spark.read.csv('ratings.csv', header=True, inferSchema=True)

#joinuje tabele z ocenami i filmami po movieId
joined_df = ratings_df.join(movies_df, on="movieId", how="inner")

#znalezienie średniej oceny dla każdego filmu
avg_ratings_df = joined_df.groupBy("title").agg(avg("rating").alias("avg_rating"))

#znalezienie filmów których średnia ocena wynosi 5.0 (jeśli średnia wynosi 5 to wiemy że każda ocena dla tego filmu była 5.0 czyli maksymalna)
max_avg_rating_movies_df = avg_ratings_df.filter(col("avg_rating") == 5.0)

#obliczenie procentu filmów, których średnia ocena wynosi 5.0
total_movies = avg_ratings_df.select("title").distinct().count()
max_avg_rating_movies_count = max_avg_rating_movies_df.select("title").distinct().count()

percentage_max_avg_rating_movies = (max_avg_rating_movies_count / total_movies) * 100

print('Procent filmów wyłącznie z maksymalną oceną:', percentage_max_avg_rating_movies)

Procent filmów wyłącznie z maksymalną oceną: 3.045580821072127


In [75]:
#8. Ktory film na najwyzsza minimalna ocene? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.

from pyspark.sql.functions import col, min, count

#wczytanie zbiorów
ratings_df = spark.read.csv('ratings.csv', header=True, inferSchema=True)
movies_df = spark.read.csv('movies.csv', header=True, inferSchema=True)

#znalezienie minimalnej oceny dla każdego filmu
min_ratings_per_movie = ratings_df.groupBy("movieId").agg(min("rating").alias("min_rating"))

#znalezienie filmu z najwyższą minimalną oceną i największą liczbą głosów
top_movie = min_ratings_per_movie.join(
    ratings_df,
    on=["movieId"],
    how="inner"
).join(
    movies_df.select("movieId", "title"),
    on="movieId",
    how="inner"
).groupBy("movieId", "title", "min_rating").agg(count("*").alias("num_votes")).orderBy(
    col("min_rating").desc(), col("num_votes").desc()
).first()

print("Film z najwyższą minimalną oceną:")
print(f"Tytuł: {top_movie['title']}")
print(f"Ocena: {top_movie['min_rating']}")
print(f"Liczba głosów: {top_movie['num_votes']}")

Film z najwyższą minimalną oceną:
Tytuł: Lamerica (1994)
Ocena: 5.0
Liczba głosów: 2


In [76]:
#9. Jaki jest rozklad ocen?
from pyspark.sql.functions import col, count

#wczytanie zbioru
ratings_df = spark.read.csv('ratings.csv', header=True, inferSchema=True)

#obliczenie rozkładu ocen
rating_distribution = ratings_df.groupBy("rating").agg(count("*").alias("count")).orderBy("rating")

print("Rozkład ocen:")
rating_distribution.show()

Rozkład ocen:
+------+-----+
|rating|count|
+------+-----+
|   0.5| 1370|
|   1.0| 2811|
|   1.5| 1791|
|   2.0| 7551|
|   2.5| 5550|
|   3.0|20047|
|   3.5|13136|
|   4.0|26818|
|   4.5| 8551|
|   5.0|13211|
+------+-----+



In [77]:
#10. Ile jest filmow zaklasyfikowanych jako dokumentalne 'documentary'?

from pyspark.sql.functions import col

#wczytanie zbioru
movies_df = spark.read.csv('movies.csv', header=True, inferSchema=True)

#obliczenie liczby filmów zaklasyfikowanych jako dokumentalne
documentary_count = movies_df.filter(col("genres").contains("Documentary")).count()

print("Liczba filmów sklasyfikowanych jako dokumenty", documentary_count)

Liczba filmów sklasyfikowanych jako dokumenty 440


In [78]:
#11. Ktory z filmow dokumentalnych z conajmniej 10 glosami ma najwysza srednia ocene?

from pyspark.sql.functions import col, count, avg

#wczytanie zbiorów
ratings_df = spark.read.csv('ratings.csv', header=True, inferSchema=True)
movies_df = spark.read.csv('movies.csv', header=True, inferSchema=True)

#znalezienie filmów dokumentalnych z co najmniej 10 ocenami
documentary_ratings = ratings_df.join(movies_df, on="movieId", how="inner").filter(col("genres").contains("Documentary"))
documentary_ratings_count = documentary_ratings.groupBy("movieId").agg(count("*").alias("num_ratings"))
documentary_with_min_10_ratings = documentary_ratings_count.filter(col("num_ratings") >= 10)

#obliczenie średniej oceny i znalezienie filmu z najwyższą średnią oceną
top_documentary = documentary_with_min_10_ratings.join(
    ratings_df,
    on="movieId",
    how="inner"
).join(
    movies_df.select("movieId", "title"),
    on="movieId",
    how="inner"
).groupBy("movieId", "title").agg(avg("rating").alias("avg_rating")).orderBy(
    col("avg_rating").desc()
).first()

# Wyświetlenie wyników
print("Film dokumentalny z min 10 głosami i najwyższą oceną średnią:")
print(f"Tytuł: {top_documentary['title']}")
print(f"Średnia ocen: {top_documentary['avg_rating']}")

Film dokumentalny z min 10 głosami i najwyższą oceną średnią:
Tytuł: Fog of War: Eleven Lessons from the Life of Robert S. McNamara, The (2003)
Średnia ocen: 4.3076923076923075


In [79]:
#13. Ile srednio kategorii przypisanych jest do 1 filmu? Ktory film ma najwiecej kategorii (+co to za kategorie)?

from pyspark.sql.functions import col, size, split, desc

#wczytanie zbioru
movies_df = spark.read.csv('movies.csv', header=True, inferSchema=True)

#obliczenie średniej liczby kategorii przypisanych do jednego filmu
avg_categories_per_movie = movies_df.withColumn("categories", size(split(col("genres"), "\\|"))) \
    .agg({"categories": "avg"}).collect()[0][0]

print("Średnia liczba kategorii przypisanych do jednego filmu:", avg_categories_per_movie)

#znalezienie filmu z największą liczbą kategorii i wyświetlenie tych kategorii
max_categories_movie = movies_df.withColumn("categories", size(split(col("genres"), "\\|"))) \
    .select("movieId", "title", "genres", "categories") \
    .orderBy(desc("categories")).first()

print("Film z największą liczbą przypisanych kategorii:")
print("ID filmu:", max_categories_movie["movieId"])
print("Tytuł:", max_categories_movie["title"])
print("Przypisane kategorie:", max_categories_movie["genres"])
print("Liczba kategorii:", max_categories_movie["categories"])

Średnia liczba kategorii przypisanych do jednego filmu: 2.2668856497639087
Film z największą liczbą przypisanych kategorii:
ID filmu: 81132
Tytuł: Rubber (2010)
Przypisane kategorie: Action|Adventure|Comedy|Crime|Drama|Film-Noir|Horror|Mystery|Thriller|Western
Liczba kategorii: 10
